In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os 
import time
from google.cloud import bigquery
import pandas_gbq
import glob

In [2]:
client = bigquery.Client()

In [3]:
query = "select * from `repositoriodedadosgpsp.cargos_lideranca.MUNIC_perfil_gestor_cultura_tipo_orgao` order by ano"  

In [4]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████|


In [6]:
df['ano'].unique()

<IntegerArray>
[2018]
Length: 1, dtype: Int64

In [19]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [25]:
df.to_csv("INEP_esforco_docente_atual_temporal.csv",index=False, encoding="latin-1", sep=';', decimal=',')

-----------------------------------------------------------

Testes

In [29]:
df = pd.read_csv("INEP_esforco_docente_atual_temporal.csv", encoding="latin-1", sep=';', decimal=',')
df

,ano,sigla_uf,ano_escolar,localizacao,dependencia_adm,docente_niveis,per_valor
0,2022,RS,Ensino Fundamental - Todos os anos,Urbana,Estadual,Nível 6,0.0
1,2022,RS,Ensino Fundamental - Todos os anos,Rural,Municipal,Nível 6,0.0
2,2022,RS,Ensino Fundamental - Todos os anos,Urbana,Estadual,Nível 6,0.0
3,2022,RS,Ensino Fundamental - Todos os anos,Urbana,Municipal,Nível 6,0.0
4,2022,SC,Ensino Fundamental - Todos os anos,Urbana,Municipal,Nível 6,13.8
...,...,...,...,...,...,...,...
3141763,2022,RS,Ensino Fundamental - Todos os anos,Urbana,Estadual,Nível 6,0.0
3141764,2022,RS,Ensino Fundamental - Todos os anos,Urbana,Privada,Nível 6,11.1
3141765,2022,RS,Ensino Fundamental - Todos os anos,Rural,Municipal,Nível 6,0.0
3141766,2022,RS,Ensino Fundamental - Todos os anos,Urbana,Municipal,Nível 6,5.9


-----------------------------------------------------------

Automatizando troca de csv

In [7]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [ ]:
## baixando os atuais com , e exportando com ";"

In [15]:
## listando todos os itens do diretório
len(glob.glob('*.csv'))

42

In [17]:
## Serve apenas na primeira vez . 

for i in list(range(0,len(glob.glob('*.csv')))):
    
    df= pd.read_csv(glob.glob('*.csv')[i])
    df.to_csv(glob.glob('*.csv')[i],index=False, encoding="latin-1", sep=';', decimal=',')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 139: invalid continuation byte